In [1]:
from __future__ import print_function, division
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch.utils.data
import torch.nn.functional as F
import time
from datasets import __datasets__
import gc
import skimage
import skimage.io
import skimage.transform
import numpy as np
from PIL import Image
from datasets.data_io import get_transform
from models.bgnet import BGNet
from models.bgnet_plus import BGNet_Plus
from utils import *
import time




In [2]:
H = 120
W = 160
N = 12

In [3]:
hg, wg = torch.meshgrid([torch.arange(0, H), torch.arange(0, W)])
hg = hg.float().repeat(N, 1, 1).unsqueeze(3) / (H-1) * 2 - 1

/disk2/users/M22_zhaoqinghao/.local/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
torch.Size(hg.size())

torch.Size([12, 120, 160, 1])

In [5]:
index = torch.arange(0,97)  # tensor([ 0 ,..., 96])
index_float = index/4.0
index_a = torch.floor(index_float)
index_b = index_a + 1
# 限制 index_a, index_b 的范围在 [0,24] 之间
index_a = torch.clamp(index_a, min=0, max= 24)
index_b = torch.clamp(index_b, min=0, max= 24)

wa = index_b - index_float
wb = index_float - index_a

# reshape wa, wb to (1,n,1,1)
wa  = wa.view(1,-1,1,1)
wb  = wb.view(1,-1,1,1)
wa = wa.float()
wb = wb.float()

In [6]:
wa.size()

torch.Size([1, 97, 1, 1])

In [7]:
# wa

In [8]:
H = 480
W = 640
N = 1

In [9]:
#读取图片
x = Image.open('/home/zhaoqinghao/DSEC/output/left/000167.png').convert('L')
right_img = Image.open('/home/zhaoqinghao/DSEC/output/right/000167.png').convert('L')
disp = Image.open('/home/zhaoqinghao/DSEC/output/disp/000167.png').convert('L')


In [10]:
xx = torch.arange(0, W).view(1, -1).repeat(H, 1)
yy = torch.arange(0, H).view(-1, 1).repeat(1, W)
xx = xx.view(1, 1, H, W).repeat(N, 1, 1, 1)
yy = yy.view(1, 1, H, W).repeat(N, 1, 1, 1)
grid = torch.cat((xx, yy), 1).float()

In [11]:
maxdisp = 3
b = N
stride = 1

In [12]:
temp_array = np.tile(np.array(range(-maxdisp + 1, maxdisp)), b) * stride
batch_shift = torch.Tensor(np.reshape(temp_array, [len(temp_array), 1, 1, 1])).cuda().requires_grad_(False)